In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
target_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/\
search?keywords=&location=Finland&geoId=100456013&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start={}"

In [ ]:
for i in range(400): 
    # TODO: 1. Figure out how many pages are exactly there (if not 400)
    #       2. Why there are so many duplicates? (Down from 9000+ to 557)

    res=requests.get(target_url.format(i))
    soup=BeautifulSoup(res.text,"html.parser")
    alljobs_on_this_page=soup.find_all("li")
    
    for x in range(0,len(alljobs_on_this_page)):
        conf = alljobs_on_this_page[x].find("div",{"class":"base-card"})
        if conf is not None:
            jobid = conf.get('data-entity-urn').split(":")[3]
            print(jobid,file=open("jobid.txt","a"))


In [52]:
with open("jobid.txt") as file:
    # jobid = file.readlines()
    # jobid = [i.strip() for i in jobid]
    # jobid =list(set(jobid))
    # jobid=[int(i) for i in jobid]
    jobid = list(set(int(line.strip()) for line in file))
print(jobid,file=open("jobid_cleaned.txt","w"))
len(jobid)

In [88]:
import langdetect

rawinfo={} # the temporary dictionary to store the raw information
result=[] # the final list to store all the scraped information

for i in range(len(jobid)):

    target_url=f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{jobid[i]}'

    resp = requests.get(target_url)
    soup=BeautifulSoup(resp.text,'html.parser')
    rawinfo["Job ID"]=f"{jobid[i]}"
    try:
        rawinfo["Company"]=soup.find("a",{"class":"topcard__org-name-link"}).text.strip()
    except:
        rawinfo["Company"]=None
    try:
        rawinfo["Job Title"]=soup.find("h2",{"class":"top-card-layout__title"}).text.strip()
    except:
        rawinfo["Job Title"]=None
    try:
        rawinfo["Level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[0].text.replace("Seniority level","").strip()
    except:
        rawinfo["Level"]=None
    try:
        rawinfo["Location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        rawinfo["Location"]=None
    try:
        rawinfo["Employment Type"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[1].text.replace("Employment type","").strip()
    except:
        rawinfo["Employment Type"]=None
    try:
        rawinfo["Job Function"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[2].text.replace("Job function","").strip()
    except:
        rawinfo["Job Function"]=None
    try:
        rawinfo["Industries"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[3].text.replace("Industries","").strip()
    except:
        rawinfo["Industries"]=None
    # TODO: Trying to add ["Salary"] column
    
    try:
        rawinfo["Job Description"] = soup.find("div", class_="description__text description__text--rich").text.strip()
        text = rawinfo["Job Description"]

        text = text.replace("Show more", "") \
            .replace("Show less", "") \
            .replace(r'\n\s\s*\n', '\n') \
            .rstrip()
            
        rawinfo["Job Description"] = text
    except:
        rawinfo["Job Description"]=None


    # text = rawinfo["Job Description"] # Define the JD for further processing
    # rawinfo["Job Description"]=text = text.replace("Show more", "").replace("Show less", "") # remove "Show more/less" text
    # rawinfo["Job Description"]=text = re.sub(r'\n\s*\n', '\n', text)
    # rawinfo["Job Description"]=text=text.rstrip()  # remove empty lines
    # print(text)

    try:
        language = langdetect.detect(text)
        if language == "fi":
            rawinfo["Posting Language"] = "Finnish"
        elif language == "en":
            rawinfo["Posting Language"] = "English"
    except:
        language = None

    try:
        if "English" and "Finnish" in text:
            rawinfo["Working Language"] = "English, Finnish"
        elif "English" in text:
            rawinfo["Working Language"] = "English"
        elif "Finnish" in text:
            rawinfo["Working Language"] = "Finnish"
        elif "suomen" and "englannin" in text:
            rawinfo["Working Language"] = "English, Finnish"
        elif "suomen" in text:
            rawinfo["Working Language"] = "Finnish"
        elif "englannin" in text:
            rawinfo["Working Language"] = "English"
    except:
        rawinfo["Working Language"] = None

    rawinfo["Job Description"] = target_url
    print(rawinfo,file=open("rawinfo.txt","a"))
    result.append(rawinfo)
    rawinfo={}




In [90]:
import csv
csv_filename = "result.csv"

with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ["Job ID", "Company", "Job Title", "Level", "Location", "Employment Type", "Job Function", "Industries", "Job Description", "Posting Language", "Working Language"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for data in range(len(result)):
        writer.writerow(result[data])




In [91]:
import pandas as pd
pd.read_csv("result.csv")

,Job ID,Company,Job Title,Level,Location,Employment Type,Job Function,Industries,Job Description,Posting Language,Working Language
0,3711559686,Mehiläinen,Hammashoitaja,Entry level,"Kemi, Lappi, Finland",Full-time,Health Care Provider,Hospitals and Health Care,https://www.linkedin.com/jobs-guest/jobs/api/j...,Finnish,NaN
1,3706544139,OMD Finland,Business Development Manager,Mid-Senior level,"Helsinki, Uusimaa, Finland",Full-time,"Business Development, Project Management, and ...",Advertising Services,https://www.linkedin.com/jobs-guest/jobs/api/j...,Finnish,NaN
2,3676297229,Validia,Psykologi,Entry level,"Helsinki, Uusimaa, Finland",Full-time,Health Care Provider,Hospitals and Health Care,https://www.linkedin.com/jobs-guest/jobs/api/j...,Finnish,NaN
3,3656744975,Cobblestone Energy,"JUNIOR TRADER - Dubai, UAE",Entry level,Tampere Metropolitan Area,Full-time,Finance and Sales,"Technology, Information and Internet",https://www.linkedin.com/jobs-guest/jobs/api/j...,English,NaN
4,3699030040,Eurooppanuoret (JEF-Finland),Secretary General,Employment type\n \n\n Full-time,"Helsinki, Uusimaa, Finland",NaN,NaN,NaN,https://www.linkedin.com/jobs-guest/jobs/api/j...,Finnish,"English, Finnish"
...,...,...,...,...,...,...,...,...,...,...,...
552,3693379572,Varjo,Technical Support Specialist,Entry level,"Helsinki, Uusimaa, Finland",Full-time,Information Technology,Computer Hardware,https://www.linkedin.com/jobs-guest/jobs/api/j...,English,"English, Finnish"
553,3687098356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/jobs-guest/jobs/api/j...,English,"English, Finnish"
554,3693393911,SOL ☀,Pakettilajittelijoita Seinäjoelle,Not Applicable,"Seinäjoki, South Ostrobothnia, Finland",Full-time,Other,Facilities Services,https://www.linkedin.com/jobs-guest/jobs/api/j...,Finnish,Finnish
555,3708196858,Turun kaupunki - Åbo stad - City of Turku,Suunnittelija,Entry level,"Turku, Southwest Finland, Finland",Full-time,"Design, Art/Creative, and Information Technology",Government Administration,https://www.linkedin.com/jobs-guest/jobs/api/j...,Finnish,"English, Finnish"
